### Welcome to the backend experimentation notebook

This notebook makes use of the clean data produced by the (cleaning experimentation notebook)[cleaning experiemental notebook.ipynb] to come up with a clustering algorithm that can be turned into the modular cluster file.